## Model 1: Entry and exit rules for backtesting - SHORT

**Model 1:** EMA50 crossover entry and trailing stop exit when in profit. 
**Entry:** After the model gives an alert and the close price has crossed down ema50 and stays below less than 14 days. When price crosses ema50 up and closes higher than ema50 for two consecutive days the entry is executed. 
**Exit:** After the entry when trailing stop price is higher than a day's low the exit is executed.

In [ ]:
import requests
import pandas as pd
# from datetime import datetime, timedelta, date
from datetime import datetime
import time
from polygon import RESTClient
import logging
import signal
import sys
import pickle
import lz4.frame  # type: ignore
import concurrent.futures
import os
import sys
import pandas as pd
import numpy as np
import glob
import nbimporter
import gzip
from modelPEG import main


In [ ]:
# Call the main function and store the returned DataFrame
# df = main()

In [ ]:
# Show dataframe
# print(df.head())

In [ ]:
# Example DataFrame setup
# df_test = pd.DataFrame({
#     'timestamp': pd.to_datetime([1,2,3,4,5,6,7,8,9,10], unit='D', origin='2022-01-01'),  # sample dates
#     'symbol': ['AA','AA','AA','AA','AA','BB','BB','BB','BB','BB'],  
#     'close': [10,9,11,11,9,100,110,120,125,150],
#     'ema50': [9,10,10.5,10,10,95,115,110,120,130],
#     'pegAlert': [1,0,0,0,0,1,0,0,0,0],
#     'trailingStop': [11,11,12,12,8,101,111,121,126,145]
# })

In [ ]:
# Example DataFrame setup
# df_test = pd.DataFrame({
#     'timestamp': pd.to_datetime([1,2,3,4,5], unit='D', origin='2022-01-01'),  # sample dates
#     'symbol': ['BB','BB','BB','BB','BB'],  
#     'close': [100,110,120,130,150],
#     'ema50': [95,115,110,120,130],
#     'pegAlert': [1,0,0,0,0]
# })

In [ ]:
def calculate_cross_below_ema50(df):    
    df['crossBelowEma50'] = 0  # Initialize the Entry column with 0

    # Reset index to ensure consecutive indexing within each symbol group
    df = df.reset_index(drop=True)

    # Iterate over rows where alert is 1
    for i, row in df[df['pegAlert'] == 1].iterrows():
        # Define the 90-row window after the alert
        end_index = min(i + 90, len(df))  # Ensure we don't go out of range
        for j in range(i + 1, end_index):
            # Check if 'close' is below 'ema50' within the 90-row window
            if df.at[j, 'close'] < df.at[j, 'ema50']:
                df.at[j, 'crossBelowEma50'] = 1  # Set 'Entry' to 1 in the row where condition is met
                break  # Stop searching within the 90-row window once condition is satisfied
    return df


In [ ]:
def calculate_entry(df):    
    # Initialize the entryPos column with 0
    df['entryPos'] = 0

    # Reset index to ensure consecutive indexing within each symbol group
    df = df.reset_index(drop=True)  

    # Initialize a counter for entries
    entry_counter = 1

    # Iterate over rows where 'crossBelowEma50' is 1
    for i, row in df[df['crossBelowEma50'] == 1].iterrows():
        # Define the 14-row window after the alert
        end_index = min(i + 14, len(df))  # Ensure we don't go out of range
        for j in range(i + 1, len(df)):
            # Check if 'close' is above 'ema50' within the 14-row window
            if df.at[j, 'close'] > df.at[j, 'ema50']:
                # The immediate next row (j + 1) remains 0
                delayed_index_3 = j + 2  # Check 3 rows after the condition is met
                
                # Check for the condition three rows after j
                if delayed_index_3 < len(df) and df.at[delayed_index_3, 'close'] > df.at[delayed_index_3, 'ema50']:
                    # Set entryPos to the current value of entry_counter for j + 3
                    df.at[delayed_index_3, 'entryPos'] = entry_counter  
                    entry_counter += 1  # Increment the counter for the next occurrence
                
                break  # Stop searching within the 14-row window once the first condition is satisfied
    return df


In [ ]:
def calculate_cross_below_ema50_for_symbols(df, symbols):
    # Initialize an empty list to hold DataFrames for each symbol
    df_list = []

    for symbol in symbols:
        # Filter the DataFrame for the current symbol
        df_symbol = df[df['symbol'] == symbol].copy()
        
        # Call the calculation function for the filtered symbol DataFrame
        df_symbol = calculate_cross_below_ema50(df_symbol)
        
        # Append the processed DataFrame to the list
        df_list.append(df_symbol)

    # Concatenate all symbol DataFrames into one final DataFrame
    df_final = pd.concat(df_list, ignore_index=True)
    
    return df_final

In [ ]:
def calculate_entry_for_symbols(df, symbols):
    # Initialize an empty list to hold DataFrames for each symbol
    df_list = []

    for symbol in symbols:
        # Filter the DataFrame for the current symbol
        df_symbol = df[df['symbol'] == symbol].copy()
        
        # Call the calculation function for the filtered symbol DataFrame
        df_symbol = calculate_entry(df_symbol)
        
        # Append the processed DataFrame to the list
        df_list.append(df_symbol)

    # Concatenate all symbol DataFrames into one final DataFrame
    df_final = pd.concat(df_list, ignore_index=True)
    
    return df_final

In [ ]:
def calculate_exit(df):    
    df['exitPos'] = 0  # Initialize the Entry column with 0

    # Reset index to ensure consecutive indexing within each symbol group
    df = df.reset_index(drop=True)

    # Initialize exit counter.
    exit_counter = 1

    # Iterate over rows where alert is 1
    for i, row in df[df['entryPos'] >= 1].iterrows():
        # Define the max df range.
        for j in range(i + 1, len(df)):
            # Check if 'trailingStop' is lower than 'high' within dataframe.
            if df.at[j, 'trailingStop'] <= df.at[j, 'high'] and df.at[j, 'close'] > df.at[j, 'open']:
                # Set entryPos to the current value of entry_counter for j + 3
                df.at[j, 'exitPos'] = exit_counter  # Add 1 to 'exitPos' in the row where condition is met
                exit_counter += 1  # Increment the counter for the next occurrence
                break  # Stop searching within the df once condition is satisfied
    return df

In [ ]:
def calculate_exit_for_symbols(df, symbols):
    # Initialize an empty list to hold DataFrames for each symbol
    df_list = []

    for symbol in symbols:
        # Filter the DataFrame for the current symbol
        df_symbol = df[df['symbol'] == symbol].copy()
        
        # Call the calculation function for the filtered symbol DataFrame
        df_symbol = calculate_exit(df_symbol)
        
        # Append the processed DataFrame to the list
        df_list.append(df_symbol)

    # Concatenate all symbol DataFrames into one final DataFrame
    df_final = pd.concat(df_list, ignore_index=True)
    
    return df_final

In [ ]:
# Call the main function and store the returned DataFrame
def load_data():
    df = main()
    return df

In [ ]:
# Main function to run the process
def entry_exit():
    df = load_data()
    # List of symbols from dataframe
    symbols_list = df['symbol'].unique()
    df = calculate_cross_below_ema50_for_symbols(df, symbols_list)
    df = calculate_entry_for_symbols(df, symbols_list) # Entry Function for all symbols.
    df = calculate_exit_for_symbols(df, symbols_list) # Exit Function for all symbols.
    
    return df

In [ ]:
df = entry_exit()
df.columns

In [ ]:
# Save the alert list to csv.
start_date = "2022-01-01"  
end_date = "2024-09-30"    
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')  # current timestamp

save_path = "C:\\Users\\SamuliMustonen\\Documents\\Ready Solutions\\Docs\\StockTrading\\Data\\entryExit"
file_name = f"peg_with_entry_exit_{start_date}_to_{end_date}_{timestamp}.csv"
full_path = f"{save_path}\\{file_name}"

# Save the DataFrame to CSV
df.to_csv(full_path, index=False)

print(f"File saved to: {full_path}")